In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wall-street-journal-2/WSJ_24.pos
/kaggle/input/wall-street-journal-2/test.words
/kaggle/input/wall-street-journal-2/hmm_vocab.txt
/kaggle/input/wall-street-journal-2/WSJ_02-21.pos


In [2]:
import string
from collections import defaultdict

In [3]:
with open("/kaggle/input/wall-street-journal-2/WSJ_02-21.pos") as f:
    train_corpus = f.readlines()

train_corpus[:5]

['In\tIN\n', 'an\tDT\n', 'Oct.\tNNP\n', '19\tCD\n', 'review\tNN\n']

In [4]:
with open("/kaggle/input/wall-street-journal-2/hmm_vocab.txt", 'r') as f:
    voc = f.read().split('\n')

print(voc[:50],'\n')
print(voc[-50:])

['!', '#', '$', '%', '&', "'", "''", "'40s", "'60s", "'70s", "'80s", "'86", "'90s", "'N", "'S", "'d", "'em", "'ll", "'m", "'n'", "'re", "'s", "'til", "'ve", '(', ')', ',', '-', '--', '--n--', '--unk--', '--unk_adj--', '--unk_adv--', '--unk_digit--', '--unk_noun--', '--unk_punct--', '--unk_upper--', '--unk_verb--', '.', '...', '0.01', '0.0108', '0.02', '0.03', '0.05', '0.1', '0.10', '0.12', '0.13', '0.15'] 

['yards', 'yardstick', 'year', 'year-ago', 'year-before', 'year-earlier', 'year-end', 'year-on-year', 'year-round', 'year-to-date', 'year-to-year', 'yearlong', 'yearly', 'years', 'yeast', 'yelled', 'yelling', 'yellow', 'yen', 'yes', 'yesterday', 'yet', 'yield', 'yielded', 'yielding', 'yields', 'you', 'young', 'younger', 'youngest', 'youngsters', 'your', 'yourself', 'youth', 'youthful', 'yuppie', 'yuppies', 'zero', 'zero-coupon', 'zeroing', 'zeros', 'zinc', 'zip', 'zombie', 'zone', 'zones', 'zoning', '{', '}', '']


In [5]:
voc.sort()
vocab={}

for i,word in enumerate(voc):
    vocab[word] = i

cnt = 0
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt>20: break

:0
!:1
#:2
$:3
%:4
&:5
':6
'':7
'40s:8
'60s:9
'70s:10
'80s:11
'86:12
'90s:13
'N:14
'S:15
'd:16
'em:17
'll:18
'm:19
'n':20


In [6]:
with open("/kaggle/input/wall-street-journal-2/WSJ_24.pos",'r') as f:
    y = f.readlines()

y[:10]

['The\tDT\n',
 'economy\tNN\n',
 "'s\tPOS\n",
 'temperature\tNN\n',
 'will\tMD\n',
 'be\tVB\n',
 'taken\tVBN\n',
 'from\tIN\n',
 'several\tJJ\n',
 'vantage\tNN\n']

In [7]:
def assign_unk(word):
    punct = string.punctuation
    noun_suffs = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
    verb_suffs = ["ate", "ify", "ise", "ize"]
    adj_suffs = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
    adv_suffs = ["ward", "wards", "wise"]
    
    if any(char.isdigit() for char in word):
        return "--unk_digit--"

    elif any(char in punct for char in word):
        return "--unk_punct--"

    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    elif any(word.endswith(suff) for suff in noun_suffs):
        return "--unk_noun--"

    elif any(word.endswith(suff) for suff in verb_suffs):
        return "--unk_verb"

    elif any(word.endswith(suff) for suff in adj_suffs):
        return "--unk_adj--"

    elif any(word.endswith(suff) for suff in adv_suffs):
        return "--unk_adv--"

    return "unk--"
    

In [8]:
def preprocess(vocab,data):
    orig = []
    prep = []

    with open(data, 'r') as f:
        for word in f:
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = assign_unk(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data, "r").readlines()))
    assert(len(prep) == len(open(data, "r").readlines()))

    return orig, prep

In [9]:
_, prep = preprocess(vocab,"/kaggle/input/wall-street-journal-2/test.words")   

In [10]:
print(len(prep))
prep[:10]

34199


['The',
 'economy',
 "'s",
 'temperature',
 'will',
 'be',
 'taken',
 'from',
 'several',
 'unk--']

In [11]:
# train_corpus[:5]
for k,v in vocab.items():
    print(f"{k}:{v}")
    cnt += 1
    if cnt>70: break

:0
!:1
#:2
$:3
%:4
&:5
':6
'':7
'40s:8
'60s:9
'70s:10
'80s:11
'86:12
'90s:13
'N:14
'S:15
'd:16
'em:17
'll:18
'm:19
'n':20
're:21
's:22
'til:23
've:24
(:25
):26
,:27
-:28
--:29
--n--:30
--unk--:31
--unk_adj--:32
--unk_adv--:33
--unk_digit--:34
--unk_noun--:35
--unk_punct--:36
--unk_upper--:37
--unk_verb--:38
.:39
...:40
0.01:41
0.0108:42
0.02:43
0.03:44
0.05:45
0.1:46
0.10:47
0.12:48
0.13:49


**POS Tagging**

In [12]:
def create_dicts(train_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    